In [21]:
import numpy as np
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size) #1行t.size列
        y = y.reshape(1, y.size) #
    # 教師データがone-hot-vectorの場合、正解ラベルのインデックスに変換
    if t.size == y.size:
        t = t.argmax(axis=1) #配列の最大値を返す（初期値１）
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t])) / batch_siz

In [22]:
t=np.arange(24)
t=t.reshape(1, t.size)
t.argmax(axis=1)

array([23])

In [37]:
y=np.random.rand(3)
print(y)
t=np.random.rand(3)
print(t)
cross_entropy_error(y, t)

[0.64814637 0.65431712 0.21354268]
[0.0491865  0.61943709 0.08539844]


0.42416315452657166

In [38]:
class SoftmaxWithLoss:
   def __init__(self):
       self.loss = None
       self.y = None # softmaxの出力
       self.t = None # 教師データ
   def forward(self, x, t):
       self.t = t
       self.y = softmax(x)
       # forwardの式
       # -sum ( t * log (y))
       self.loss = cross_entropy_error(self.y, self.t)
       return self.loss
   def backward(self, dout=1):
       # backwardの式
       # yi - ti (iはIndex)
       batch_size = self.t.shape[0]
       dx = (self.y - self.t)/batch_size# Backwardを実装して、微分値をdxに代入してください
       return dx

In [39]:
x = np.random.rand(5)
print("x:{}".format(x))
t = np.array([0, 0, 0, 1, 0])
print("t:{}".format(t))
softmaxWithLoss = SoftmaxWithLoss()
loss =softmaxWithLoss.forward(x, t)
back =softmaxWithLoss.backward(dout=1)
print("loss:{}".format(loss))
print("back:{}".format(back))

x:[0.78737373 0.40794389 0.99743192 0.16772942 0.81269579]
t:[0 0 0 1 0]
loss:2.1196651553080024
back:[ 0.0446251   0.03053479  0.05505621 -0.17598563  0.04576953]


出力例
一回目：x:[0.18729076 0.94969355 0.1658287  0.97256373 0.00519878]
t:[0 0 0 1 0]
loss:1.1816599317949252
back:[ 0.02797705  0.05996657  0.027383   -0.13864618  0.02331956]

二回目：x:[0.1923468  0.88244657 0.74416427 0.76435631 0.37221984]
t:[0 0 0 1 0]
loss:1.469108196513353
back:[ 0.02597669  0.0517953   0.04510608 -0.15397388  0.0310958 ]

１回目の入力値と2回目の入力値の教師データを比較すると、１回目の入力値の方が大きいので、誤差が少ないことがわかる。

Ｒｅｌｕ　と　Ａｆｆｉｎｅの実装

In [40]:
class Relu:
    def __init__(self):
        self.mask = None
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx

In [41]:
class Affine:
    def __init__(self, W, b):
        self.W =W
        self.b = b
        self.x = None
        self.original_x_shape = None
        # 重み・バイアスパラメータの微分
        self.dW = None
        self.db = None
    def forward(self, x):
        # テンソル対応
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x
        out = np.dot(self.x, self.W) + self.b
        return out
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        dx = dx.reshape(*self.original_x_shape)  # 入力データの形状に戻す（テンソル対応）
        return dx

two layer netにおける勾配の確認

In [2]:
import numpy as np
from dataset.mnist import load_mnist
# データの読み込み
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)
x_batch = x_train[:3]
t_batch = t_train[:3]
# 数値微分
grad_numerical = network.numerical_gradient(x_batch, t_batch)
# Backward
#grad_backprop = gradient(x_batch, t_batch)
grad_backprop = gradient(network, x_batch, t_batch)
for key in grad_numerical.keys():
   diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
   print(key + ":" + str(diff))

URLError: <urlopen error [Errno -3] Temporary failure in name resolution>

In [3]:
def gradient(network, x, t):
   # 自分で実装したSoftmax with lossクラスを使ってみてください
   lastLayer = SoftmaxWithLoss()
   # forward
   #self.loss(x, t)
   network.loss(x, t)
   # backward
   dout = 1
   dout = lastLayer.backward(dout)
   #layers = list(self.layers.values())
   layers = list(network.layers.values())
   layers.reverse()
   for layer in layers:
      dout = layer.backward(dout)
   # 設定
   grads = {}
   #grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
   grads['W1'], grads['b1'] = network.layers['Affine1'].dW, self.layers['Affine1'].db
   #grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db
   grads['W2'], grads['b2'] = network.layers['Affine2'].dW, self.layers['Affine2'].db
   return grads

In [4]:
import numpy as np
from dataset.mnist import load_mnist
# データの読み込み
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)
iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1
train_loss_list = []
train_acc_list = []
test_acc_list = []
iter_per_epoch = max(train_size / batch_size, 1)
for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    # 勾配
    #grad = network.numerical_gradient(x_batch, t_batch)
    #grad = gradient(x_batch, t_batch)
    grad = gradient(network, x_batch, t_batch)
    # 更新
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

URLError: <urlopen error [Errno -3] Temporary failure in name resolution>

感想:全体的にエラーが多くて学習が進まない。早めに課題を始めてわからないところは質問するべきだと感じた。

In [ ]:
参考文献：ゼロから始めるDeepLearning　第５章「誤差逆伝播法」